<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Гибкий биополимер – модель моста из пучка волокон

### Examples – Biophysics
<section class="post-meta">
By Jonas Tjemsland, Andreas S. Krogen and Jon Andreas Støvneng.
</section>
Last edited: January 31st 2019
___
This notebook is based on an assignment given in the course TMA4320 *Introduction to Scientific Computation* at NTNU in 2017. The assignment was prepared by Alex Hansen, Jon Andreas Støvneng
Jonas Blomberg Ghini and Rita Dias. The codes are based on a set of python scripts (see the references) originally developed by Håkon Tormodsen Nygård, later modified by Jonas Blomberg Ghini.
___

## Вступление

Представьте себе полимер, который приклеен к поверхности. Это, конечно, связано с молекулярными силами. Предположим, что полимерные компоненты содержат электрические заряды или дипольный момент. Затем, используя достаточно сильное электрическое поле, мы можем оторвать полимер от поверхности. Молекулы воды также обладают дипольным моментом, и можно согнуть струю воды из крана с помощью расчески, заряженной статическим электричеством. Это тот же самый эффект. Цель состоит в том, чтобы смоделировать это распутывание. Если полимер ограничен поверхностью, это означает, что через регулярные (или нерегулярные) промежутки времени вдоль полимера существуют *функциональные группы*, которые могут образовывать более или менее прочные связи с атомами на поверхности. Эта связь может быть описана потенциальной функцией $E(R)$, которая обычно выглядит примерно так, как потенциал, показанный на рисунке 1 (подробнее об этом вы можете прочитать в нашей записной книжке про [Потенциал Леннарда-Джонса](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/lennard_jones_potential.ipynb)). Около равновесия потенциал приблизительно гармонический, $E(R)=E_0 + \frac{1}{2} k(R-R_0)^2$, и можно оценить силу, необходимую для разрыва текущей связи между поверхностью и полимером. 

<img src="images/Argon_dimer_potential.png" width="400">

**Рисунок 1.** *Типичный потенциал, описывающий химическую связь. Вблизи минимума потенциал примерно гармонический.*

Проблема, описанная выше, полностью аналогична макроскопическому кабелю, который крепится к потолку с помощью крючков, где крючки рассматриваются как идеальные пружины. Кабель представляет собой полимер, гравитационное притяжение представляет собой электрическое поле, а пружины представляют собой связи, прикрепляющие полимер к поверхности. Модель кабеля обеспечивает классическую макроскопическую картину нашей задачи. Постепенно увеличивающееся электрическое поле, например, может быть аналогично кабелю с увеличивающимся весом из-за коррозии.

## Провисающий кабель

Мы опишем вариант *Модели пучка волокон* (см., например, Ссылку [[1]](#rsc) для общего введения), называемый *Моделью моста пучка волокон*, впервые изученной Нюгордом [[2]](#rsc). Представьте себе кабель, свисающий параллельно потолку на  наборе равноудаленных крючков (рис. 2).

<img src="images/cable_model.png" width="400">

**Рисунок 2.** *Кабель, подвешенный к потолку на одинаково удаленных крючках.*

Предположим, что концы кабеля сварены вместе, создавая кольцо (периодические граничные условия). Более того, предположим, что это кольцо "большое" (то, что мы подразумеваем под большим, станет очевидным позже).

### Кабель

Теперь мы перейдем к описанию самого кабеля. Мы вводим систему координат с направлением $x$ вдоль кабеля и вертикальной осью $y$ с $y=0$ на потолке (рис. 2). Затем кабель может быть описан функцией $y(x)$. Предположим, что кабель имеет равномерное распределение массы, с массой $\lambda$ на единицу длины. Сила, действующая на кабель из-за гравитационного притяжения, тогда

$$f_g=-\lambda g,$$

на единицу длины. Мы предполагаем, что кабель обладает такой жесткостью, что он создает внутренние силы, противодействующие изгибу (как нормальное напряжение, так и напряжение сдвига). До тех пор, пока изгибы малы, чистое усилие на единицу длины из-за напряжения определяется 

$$f_b=-B\frac{\text d^4y}{\text dx^4}$$

(см., например, Ссылку [[3]](#rsc)). Здесь $B=EI$ - жесткость на изгиб, задаваемая произведением модуля упругости (или модуля Юнга) $E$ и момента инерции области (или второго момента участка) $I$. В случае кабеля с круглым поперечным сечением мы имеем $I=\pi r^4/4$.

Для стали у нас $E\simeq 200 \text{ ГПа}$ (Структурная ASTM-A36 в [[4]](#rsc)). ДНК имеет модуль Юнга порядка $0.3–1\;\text{ГПа}$ (мы будем использовать $0.3\;\text{ГПа}$), что эквивалентно твердому пластику [[5]](#rsc).

Как упоминалось ранее, мы будем сваривать концы кабеля вместе, создавая круг. Предполагается, что этот круг достаточно велик, чтобы силой изгиба можно было пренебречь.

### Крючки

Теперь мы подошли к описанию крючков. Они расположены по периметру кольца на расстоянии $L$ друг от друга (измеряется вдоль кольца). Всего существует $N$ крючков, пронумерованных $k=0, 1, ... N-1$. Начало координат $x=0$ расположено таким образом, что позиция $k$-го крючка равна $x_k=Lk$. Как упоминалось выше, мы рассматриваем периодические граничные условия (из-за кольца). Таким образом, крючок $k$ примыкает к крючкам $k-1$ и $k+1$ для $k = 1,2,..., N-2$. Крючок $0$ примыкает к $1$ и $N-1$, а крючок $N-1$ к $N-2$ и $0$.

Каждый из крючков аппроксимируется идеальной пружиной с постоянной жесткости $\kappa_k$. Другими словами, потенциал является гармоническим, и пружины следуют закону Гука.

Крючок ломается при увеличении усилия. Таким образом, константа пружины может иметь два разных значения: $\kappa_k=\kappa>0$ для неповрежденной пружины и $\kappa_k=0$ для сломанной пружины. Мы вернемся к этому позже.

Сила, действующая на кабель на единицу длины от крюка $k$, равна

$$f_k=-\kappa_k(y(x)-y_0)\delta(x-Lk),$$

где $|y_0|$ - длина пружин в состоянии равновесия. Помните, что и $y(x) < 0$, и $y_0<0$ в нашей системе координат. Сила от крюка $k$, таким образом, направлена *вверх* ($+y$-направление). Функция $\delta(x-Lk)$ является *дельта-функцией Дирака*.

### Уравнение кабеля

Мы будем рассматривать кабель в состоянии покоя. По первому закону Ньютона сумма сил на кабеле должна быть равна нулю:

$$f_b+f_g+f_k = 0.$$

Объединив все приведенные выше уравнения, мы получим

\begin{equation}
B\frac{\text d^4y(x)}{\text dx^4} = -g\lambda - \sum_{k=0}^{N-1}\kappa_k(y(x)-y_0)\delta(x-lk). 
\label{one} \quad (1)
\end{equation}

Это уравнение *кабеля* и описывает форму кабеля $y(x)$. 

## Решение уравнения кабеля

Введем безразмерные переменные $\xi \equiv x/L$ и $\eta \equiv y/L$ и безразмерные параметры $\alpha\equiv g\lambda L^3/B$ и $\beta_k\equiv \kappa_kL^3/B$. Если мы теперь сместим начало координат оси $y$ на $y_0$, мы сможем записать (1) как

\begin{equation}
\frac{\text d^4\eta(\xi)}{\text d\xi^4} = -\alpha - \sum_{k=0}^{N-1}\beta_k\eta(\xi)\delta(\xi-k). 
\label{two} \quad (2)
\end{equation}

На интервале $k<\xi<k+1$ уравнение (2) сводится к

\begin{equation}
\frac{\text d^4\eta(\xi)}{\text d\xi^4} = -\alpha.
\label{three} \quad (3)
\end{equation}

Это уравнение легко решить

\begin{equation}
\eta_k(\xi)=-\frac{\alpha}{24}(\xi-k)^4 + a_k(\xi - k)^3 + b_k(\xi-k)^2 + c_k(\xi-k)+d_k,
\label{four} \quad (4)
\end{equation}

где $\eta_k(\xi)$ является решением (3) - и, следовательно, (2) - на интервале $k<\xi<k+1$. Коэффициенты $a_k, b_k, c_k, d_k$ являются константами интегрирования, которые еще предстоит определить.

Давайте теперь рассмотрим $\xi=k$. "Справа" от $\xi=k$ у нас есть решение $\eta_k(\xi)$, а "слева" у нас есть решение $\eta_{k-1}(\xi)$. Интегрируя (2) на интервале $\xi\in[k-\epsilon, k+\epsilon]$ для некоторого небольшого $\epsilon$, мы получаем

$$\frac{\text d^3 \eta_k (k+\epsilon)}{\text d\xi^3} - \frac{\text d^3 \eta_{k-1} (k-\epsilon)}{\text d\xi^3}=-2\epsilon \alpha-\beta_k\eta_k(k).$$

Если мы устремим $\epsilon\to 0$, это приведет к

\begin{equation}
\frac{\text d^3 \eta_k (k)}{\text d\xi^3} - \frac{\text d^3 \eta_{k-1} (k)}{\text d\xi^3}=-\beta_k\eta_k(k).
\label{five} \quad (5)
\end{equation}

Вставляя решение (4), мы получаем первое граничное условие.

Уравнение (5) говорит, что третья производная от $\eta$ имеет конечный шаг при $\xi=k$. В свою очередь, это означает, что вторая и первая производная от $\eta$, а также сама $\eta$ непрерывны при $\xi=k$. Это приводит к трем дополнительным граничным условиям:

\begin{align}
\frac{\text d^2 \eta_k (k)}{\text d\xi^2} - \frac{\text d^2 \eta_{k-1} (k)}{\text d\xi^2}&=0 \\
\frac{\text d \eta_k (k)}{\text d\xi} - \frac{\text d \eta_{k-1} (k)}{\text d\xi}&=0 \\
\eta_k(k)-\eta_{k-1}(k)&=0.
\label{six} \quad (6)
\end{align}

Используя четыре граничных условия (5)-(6) с решением (4) , получаем четыре уравнения, которые связывают константы $a_k, b_k, c_k, d_k$ by $a_{k-1}, b_{k-1}, c_{k-1}, d_{k-1}$. Мы получаем систему уравнений для $k=0, 1, ..., N-1$.

Как всегда, мы начинаем с импорта пакетов, установки общих параметров рисунка и определения физических параметров.

In [ ]:
import matplotlib.pyplot as plt
from scipy.linalg.lapack import ssbevd
import numpy as np
from matplotlib import animation
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
from scipy.sparse.linalg import spsolve
from scipy import sparse
import matplotlib.gridspec as gridspec
import progressbar
import time

In [ ]:
newparams = {'axes.labelsize': 25, 'axes.linewidth': 1, 'savefig.dpi': 200,
             'lines.linewidth': 3, 'figure.figsize': (20, 10),
             'ytick.labelsize': 25, 'xtick.labelsize': 25,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,
             'figure.titlesize': 25, 
             'legend.fontsize': 25, 'legend.frameon': True, 
             'legend.handlelength': 1.5, 'axes.titlesize': 30,
             'axes.titlepad' : 20,
             'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)

hbar = 1.05e-34  # J⋅s.   Reduced Plank's constant
m = 9.11e-31     # kg.    Electron mass
e = 1.6e-19      # C.     Elementary cahrge
g = 9.81         # m/s².  Gravitational acceleration

global N, lengthBetweenFibers, unitOfSeparation, alpha, beta_base, beta, thresholdExtensions
global A, u, x

### Некоторые параметры

Мы будем рассматривать стальной кабель с массовой плотностью $\rho_\text{сталь}=8030\text{ кг/м}^3$, а также молекулу ДНК. Чтобы быстро переключаться между ними, мы создаем некоторые функции, которые возвращают соответствующие параметры.

Для молекулы ДНК мы сделаем следующие предположения: Молекула ограничена поверхностью (например, золотом) с привязкой для каждого 5-го нанометра (нм). Каждое связующее может быть аппроксимировано как идеальная пружина с равновесной длиной 0.5 нм и постоянной жесткости 0.1 Н/м. Молекула имеет приблизительное круглое поперечное сечение радиусом 1 нм. Молекула обычно заряжена отрицательно (в ее *"естественной среде"*), примерно с двумя элементарными зарядами на 0.34 нм. Предположим, что этот заряд равномерно распределен вдоль молекулы. Кроме того, предположим, что заданное связывание между молекулой и поверхностью может выдержать примерно полосу длиной 0.1 нм до разрыва.

In [ ]:
def get_DNA_params(N):
    
    electricFieldStrength = 5.0e4  # V/m
    
    # Radius of the hanging cable
    r_DNA = 1e-9  # m
    # Distance between hooks/fibers
    l_DNA = 5e-9  # m
    # Young's modulus
    E_DNA = 300e6 # Pa
    # Charge per length
    charge_lambda = 2 * e / 0.34e-9    # C/m
    # Cable stiffness
    B = E_DNA * np.pi / 4 * r_DNA ** 4 # N⋅m²
    
    alpha = electricFieldStrength * charge_lambda * l_DNA ** 3 / B
    # Conversion factor between alpha and real load, here: electric field (2.0e6 N/C (V/m)
    # for reference values)
    alphaToLoadConversionFactor = B / charge_lambda / l_DNA ** 3
    
    # Spring constant of the dipole bindings
    kappa = 0.1 # N/m
    beta_base = kappa * l_DNA ** 3 / B
    # Make it a vector, so that updating the value is possible as fibers/bindings are broken
    beta = np.ones(N) * beta_base
    
    # Allowable extension in m
    thresholdExtension_base = 0.1e-9  # m
    # Conversion for direct comparison with eta(xi_minus_k)
    thresholdExtension_normalized = thresholdExtension_base / l_DNA
    # Select N random fiber threshold extensions
    thresholdExtensions = np.random.uniform(0, thresholdExtension_normalized, N)
    
    # Make tuple of load type, its unit, its relation to SI, the model's lambda parameter
    # and ITS unit
    loadType_and_units = ('Electric field strength', 'MV/m', 1e6, charge_lambda, 'C/m')
    
    return (loadType_and_units, alphaToLoadConversionFactor, beta_base,
            thresholdExtension_normalized)


def get_steel_params(N):

    # Radius of the hanging cable
    r_cable = 0.01 # m
    # Distance between hooks/fibers
    l_cable = 1    # m
    # (3.14e-4 m^2 for reference values)
    crossSection_cable = np.pi * r_cable ** 2 # m²
    # Mass density of steel
    RHO_STEEL = 8030 # kg/m³
    # Mass per length
    mass_lambda = RHO_STEEL * crossSection_cable # kg/m
    # Radius of the wire holding up the large cable
    r_wire = 0.0005 # m
    # Length of fibers/wires, m
    l_wire = 0.5    # m
    # (7.854e-7 m^2 for reference values)
    crossSection_wire = np.pi * r_wire ** 2
    
    # Young's modulus
    E_cable = 200e9 # Pa
    # Young's modulus
    E_wire = 200e9  # Pa
    
    # Cable stiffness
    B = E_cable * np.pi / 4 * r_cable ** 4      # N⋅m²
    # Spring constant of the fibers/steel wires
    kappa = E_wire * crossSection_wire / l_wire # N/m    
    
    alpha = g * mass_lambda * l_cable ** 3 / B
    # Conversion factor between alpha and real load, here: electric field (160.0 kg/m for
    # reference values)
    alphaToLoadConversionFactor = B / g / l_cable ** 3
    
    # 200 for reference values
    beta_base = kappa * l_cable ** 3 / B
    # Expressed as a vector to allow for different effectively different spring constants,
    # if ever needed
    beta = beta_base * np.ones(N)
    
    # For steel, from Wikipedia, 250 - 2500 MPa
    yieldStrength = 800e6
    # Dividing by the cable span makes the extension directly comparable to eta
    thresholdExtension_normalized = (yieldStrength * crossSection_wire / kappa) / l_cable
    # Randomly select a threshold extension for each fiber
    thresholdExtensions = np.random.uniform(0, thresholdExtension_normalized, N)
    
    # Make tuple of load type, its unit, its relation to SI, the model's lambda parameter and
    # ITS unit
    loadType_and_units = ('Mass density', 'kg/m', 1, mass_lambda, 'kg/m')
    
    return (loadType_and_units, alphaToLoadConversionFactor, beta_base,
            thresholdExtension_normalized)


def get_artificial_params(N):
    alpha = 1
    alphaToLoadConversionFactor = 1
    
    beta_base = 1
    # Expressed as a vector to allow for different effectively different spring
    # constants, if ever needed
    beta = beta_base * np.ones(N)
    
    thresholdExtension_normalized = 1
    # Randomly select a threshold extension for each fiber
    thresholdExtensions = np.random.uniform(0, thresholdExtension_normalized, N)

    # Make tuple of load type, its unit, its relation to SI, the model's lambda parameter and
    # ITS unit
    loadType_and_units = ('Arbitrary system load', '[a.u.]', 1, 1, '[a.u.]')
    
    return (loadType_and_units, alphaToLoadConversionFactor, beta_base,
            thresholdExtension_normalized)

### Матричное уравнение

Определив вектор

$$\vec x = [a_0, b_0, c_0, d_0, ... a_k, b_k, c_k, d_k, ... a_{N-1}, b_{N-1}, c_{N-1}, d_{N-1}]^T,$$

набор уравнений (полученный с помощью четырех граничных условий (5)-(6) с решением (4)) может быть сформулирован как матричное уравнение

$$A\vec x = \vec u.$$

Таким образом, нам нужны функции для создания правой части $\vec u$ и матрицы $A$ для заданного $N$.

In [ ]:
def make_rhs_vec(N):
    """
    Make the right hand side of the system, called u in the matrix equation
    :param N: System size
    :return u: the rhs of the matrix equation to be solved
    """
    u = np.zeros(4 * N, dtype=np.float64)
    u[0::4] = -1.
    u[1::4] = -1 / 2.0  
    u[2::4] = -1 / 6.0
    u[3::4] = -1 / 24.0
    return u

Мы будем использовать разреженный решатель для матричных систем. Использование разреженного матричного представления имеет несколько преимуществ: мы не храним большое количество нерелевантных нулей, а время вычислений сокращается за счет исключения операций с нулевыми элементами. В Python существует несколько способов обработки разреженных матриц. Взгляните на [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html)! Следующая функция создает [Dictionary Of Keys based sparse matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.dok_matrix.html) из матрицы $A$.

In [ ]:
def make_dok_matrix(N, beta=1.0):
    """
    With a given system size and fiber strength, create a sparse matrix to be used with
    a sparse solver.
    :param N: Number of fibers
    :param beta: Normalized spring constant of intact fibers
    :return A_sparseDok: The sparse Dictionary of Keys matrix A_sparseDok
    """
    # By using the dok_matrix, assigning elements is exactly like for a dense matrix
    A_sparseDok = sparse.dok_matrix((4 * N, 4 * N))
    # Index to keep track of the elements called "k"
    ind = 0
    # Index to keep track of the elements called "k-1"
    ind_minus = 4 * (N - 1)

    for i in np.arange(N):
        # Eq. 1
        A_sparseDok[ind + 0, ind + 0] = 6.0
        A_sparseDok[ind + 0, ind + 3] = beta
        A_sparseDok[ind + 0, ind_minus + 0] = -6.0
        # Eq. 2
        A_sparseDok[ind + 1, ind + 1] = 2.0
        A_sparseDok[ind + 1, ind_minus + 0] = -6.0
        A_sparseDok[ind + 1, ind_minus + 1] = -2.0
        # Eq. 3
        A_sparseDok[ind + 2, ind + 2] = 1.0
        A_sparseDok[ind + 2, ind_minus + 0] = -3.0
        A_sparseDok[ind + 2, ind_minus + 1] = -2.0
        A_sparseDok[ind + 2, ind_minus + 2] = -1.0
        # Eq. 4
        A_sparseDok[ind + 3, ind + 3] = 1.0
        A_sparseDok[ind + 3, ind_minus + 0] = -1.0
        A_sparseDok[ind + 3, ind_minus + 1] = -1.0
        A_sparseDok[ind + 3, ind_minus + 2] = -1.0
        A_sparseDok[ind + 3, ind_minus + 3] = -1.0

        # Advance the indices for the next iteration
        ind = 4 * (i + 1)
        ind_minus = 4 * i

    return A_sparseDok

### Простой аналитический пример

Легко найти решение для набора уравнений, когда все крючки идентичны, например, $\beta_k=\beta$. Из-за симметрии у нас будет $a_k=a, b_k=b, c_k=c, d_k=d$, и уравнения (5) и (6) имеют решение $a=\alpha/12$, $b=-\alpha/24$, $c=0$, $d=-\alpha/\beta$. Давайте попробуем решить $A\vec x=\vec u$ численно, чтобы увидеть, получим ли мы такое же решение, и построим результирующее $\eta(\xi)$, чтобы проверить, кажется ли это разумным. Давайте использовать $N=5$ и $\alpha=\beta=1$.

In [ ]:
N = 5
loadType_and_units, alpha, beta_base, thresholdExtension_normalized = get_artificial_params(N)
A_sparseDok = make_dok_matrix(N, beta_base)
u = make_rhs_vec(N)*alpha
# Solve the system with the sparse solver of SciPy
x = spsolve(A_sparseDok.tocsc(), u)
# Print the solution for a, b, c and d
print("a = %.3f = 1/%.3f"%(x[0], 1/x[0]))
print("b = %.3f = 1/%.3f"%(x[1], 1/x[1]))
print("c = %.3f"%(x[2]))
print("d = %.3f"%(x[3]))
# Plot results
xi = np.linspace(0.0, 1.0, 200)
for k in range(N):
    # Solution polynomial, eq. (11)
    eta = -(alpha/24)*xi**4 + x[4*k + 0]*xi**3 + x[4*k + 1]*xi**2 + x[4*k + 2]*xi + x[4*k + 3]
    # Plot the cable location
    plt.plot(xi + k, eta)
title = (r"$\alpha = \frac{\lambda g L^3}{B} = %.2f,\quad \beta = \frac{\kappa L^3}{B} = %.2f$"
         %(alpha, beta_base))
plt.title(title)
plt.xlabel(r"$x/L$")
plt.ylabel(r"$y(x)/L$")
plt.show()

**Небольшое упражнение:** Нарисуйте $\text d\eta /\text d\xi$, $\text d^2\eta /\text d\xi^2$ и $\text d^3\eta /\text d\xi^3$ и проверьте, выполнены ли граничные условия.

## Распутывание

Пусть каждый крючок (или химическая связь) имеет заданное максимальное натяжение, которое он может выдержать, прежде чем сломается. То есть мы делаем следующее предположение:

$$\beta_k = \begin{cases}
\beta & \mbox{if $|\eta(k)| < t_k$,}\\
0 & \mbox{if $|\eta(k)| \geq t_k$,}
\end{cases}$$

где $t_k$ - порог разрыва для крючка $k$. Для простоты мы позволим каждому $t_k$ быть случайным числом между 0 и заданным максимальным значением $t_\text{max}$. Это даст равномерное пороговое распределение, но мы, конечно, могли бы выбрать какое-то другое распределение (может быть, нормальное распределение было бы лучшим выбором?).

Представьте, что значение $\alpha$ постепенно увеличивается. Физически это может быть описано кабелем с увеличивающимся весом из-за коррозии (или увеличения электрического поля в случае полимера). Более драматичной ситуации соответствовала бы система тушения с полыми трубами, вес которых увеличивается по мере заполнения их водой во время пожарной сигнализации! Тем не менее, при все более тяжелом кабеле крючки начнут ломаться. Как только крюк $k$ сломается, $\beta_k$ станет равным нулю. Это изменяет нашу матрицу $A$, и нам нужно снова решить матричную систему, чтобы найти правильный $\vec x$ и, следовательно, $\eta(\xi)$. Следующая функция обновит нашу (разреженную) матрицу $A$, когда данный крючок будет разорван.

In [ ]:
def break_fiber_in_sparse_dok(A_sparseDok, indexOfBrokenFiber):
    """
    Break a fiber in the system by setting the corresponding matrix element to zero.

    :param A_sparseDok: Sparse Dictionary of Keys matrix
    :param indexOfBrokenFiber: What fiber to break next (type: INT)
    :return A_sparseDok: Same matrix as input, but with one more fiber broken
    """
    indexInMatrix = 4 * indexOfBrokenFiber
    A_sparseDok[indexInMatrix + 0, indexInMatrix + 3] = 0.0
    return A_sparseDok

**Небольшое упражнение:** Сломайте один или несколько крючков в настройке предыдущего раздела и нанесите соответствующий $\eta(\xi)$.

Самый простой способ выполнить схему, упомянутую выше, - шаг за шагом увеличивать $\alpha$ и смотреть, что происходит с кабелем и крючками для каждого шага. Однако есть более разумный метод: можно использовать тот факт, что система линейна.

Правая часть нашего матричного уравнения, $A\vec x = \vec u,$ может быть записана как $\alpha \vec u_\text{ref},$ где $\vec u_\text{ref}$ - это некоторая *"эталонная нагрузка"* с $\alpha=1$ с соответствующим решением $\vec x_\text{ref}$. Это означает, что $\vec x = \alpha \vec x_\text{ref}$. Предположим, что следующий крючок, который обрывается, - это $j$. Это означает, что мы оцениваем решение (4) в позиции крючка $\xi=j$ непосредственно перед разрывом крючка. Из (4) ясно, что $\eta_j(j) = d_j$. Более того, это означает, что $d_j$ в этот момент (непосредственно перед разрывом крючка) равен отрицательному порогу $-t_j$. Линейность в $\alpha$, описанная выше, подразумевает, что $d_j = \alpha d_\text{ref j}$ и

$$\frac 1\alpha = -\frac{d_\text{ref j}}{t_j}.$$

Введем соотношение 

$$r_k = \left(-\frac{\beta_k\eta(k)}{t_k}\right)_{\alpha=1}$$

для всех крючков $k\in[0, N-1]$. Обратите внимание, что $r_k$ - это, по сути, соотношение между растяжением и пороговым растяжением. Таким образом, крючок, который сломается следующим, - это крючок с наибольшим значением $r_k$! $\beta_k$ включен в $r_k$, чтобы гарантировать, что мы не выберем крючок, который уже сломан (помните, что $\beta_k=0$ для сломанного крючка).

Когда мы определим, какой крючок $j$ сломается следующим, мы сможем вычислить соответствующее значение $\alpha$. Объединив два предыдущих уравнения, мы получим

$$\alpha = \frac{\beta}{r_j}.$$

Давайте подведем итог: мы начинаем с того, что все крючки в целости и сохранности. Мы находим $k$ для крючка, который ломается первым, и соответствующее значение $\alpha$. Теперь установим значение $\beta_k$ равным нулю и снова решим матричное уравнение. Повторяем, чтобы найти всю последовательность $\alpha_n$, $1\leq n\leq N$.

Этот алгоритм реализован в приведенной ниже функции. 

In [ ]:
def run_simulation(N, beta, thresholdExtension_base):
    """
    This function is the bread and butter of the project, in that it incorporates the
    strain-curve algorithm, and returns the order and weights (alpha's) related to each
    breaking of a fiber.

    :param N: System size
    :param beta: Normalized spring constant (single value, not vector)
    :param thresholdExtension_base: Baseline number for the thresholds, to be used in
                                    a random selection
    :param simID: Name of the simulation, needed only to keep track of progress

    :return breaking_order: Vector with the order in which the fibers broke in this
                            particular simulation
    :return breaking_alpha: Vector with the alpha's needed to break each successive fiber
    """
    
    thresholds = np.random.uniform(0, thresholdExtension_base, N)

    # Initialize the system, assuming all fibers are intact
    A_sparseDok = make_dok_matrix(N, beta)
    u = make_rhs_vec(N)

    # beta_values = beta for elements which are broken and zero otherwise
    beta_values = np.zeros(N) + beta

    # Initialize the two return values of the function
    breaking_order = np.zeros(N)  # Stores the order in which the fibers break
    breaking_alpha = np.zeros(N)  # Stores the alpha necessary to break the particular fiber

    # The weakest fiber is always the first to break
    # Get the "name" of the weakest fiber
    indexOfFiberToBreak = np.argmin(thresholds)
    # Set the corresponding beta value to zero
    beta_values[indexOfFiberToBreak] = 0.0
    # Break the corresponding value in the matrix
    A_sparseDok = break_fiber_in_sparse_dok(A_sparseDok, indexOfFiberToBreak)
    # Again, the weakest fiber always breaks first
    breaking_order[0] = indexOfFiberToBreak
    breaking_alpha[0] = beta * thresholds[indexOfFiberToBreak]

    for i in np.arange(N - 1):
        # Solve the system with the sparse solver of SciPy
        x = spsolve(A_sparseDok.tocsc(), u)

        # Check which fiber to break next
        allExtensions = -x[3::4]
        # Calculate the ratios
        ratioExtensionToThreshold = beta_values * (allExtensions / thresholds)
        # Find the largest extension (a large extension means a large ratio of this
        # extension compared with the threshold)
        indexOfFiberToBreak = np.argmax(ratioExtensionToThreshold)
        # Set that fiber's beta to zero
        beta_values[indexOfFiberToBreak] = 0.0
        # Update the order of breaks
        breaking_order[i + 1] = indexOfFiberToBreak
        # Calculate the alpha necessary for this fiber to break
        breaking_alpha[i + 1] = beta / ratioExtensionToThreshold[indexOfFiberToBreak]
        # Update the sparse matrix, in preparation for the next iteration
        A_sparseDok = break_fiber_in_sparse_dok(A_sparseDok, indexOfFiberToBreak)

    return breaking_order, breaking_alpha

### Запуск моделирования

Все, что остается, - это многократно выполнить моделирование, усреднить результаты и построить графики. Давайте создадим для этого основную функцию! Сделанные графики более подробно описаны ниже.

In [ ]:
def main(what_to_do, numberOfSimsToRun, N):
    
    # Selection structure for the various model parameters, corresponding to different
    # kinds of cables
    if what_to_do == 'DNA':
        params = get_DNA_params(N)
    elif what_to_do == 'steel':
        params = get_steel_params(N)
    elif what_to_do == 'artificial':
        params = get_artificial_params(N)
    else:
        raise ValueError('Supported simulation types: DNA, steel, artificial')

    loadType_and_units, alphaToLoadConversionFactor, beta, thresholdExtension = params
    
    # Unpack the "loadType_and_units" tuple and assign elements to variables
    (loadName, loadName_unit, loadName_unit_order, modelLambda, 
                                     modelLambda_unit) = loadType_and_units

    bar = progressbar.ProgressBar()
    breakingOrder = []
    alphaForEachFiberBreaking = []
    # Run sims
    for simID in bar(range(numberOfSimsToRun)):
        # Store the new values in temporary vectors
        breakingOrder_new, alphaForEachFiberBreaking_new = run_simulation(N, beta,
                                                                          thresholdExtension)
        # Append the new breakingOrder vector to the old one, growing the old one
        breakingOrder = np.append(breakingOrder, breakingOrder_new)
        # Same here
        alphaForEachFiberBreaking = np.append(alphaForEachFiberBreaking,
                                              alphaForEachFiberBreaking_new)

    # Once all the simulations are done, we have to reshape the results, to a matrix,
    # rather than long appended lists, so that we can manipulate them more easily
    breakingOrder = breakingOrder.reshape((numberOfSimsToRun, N))
    # Same here
    alphaForEachFiberBreaking = alphaForEachFiberBreaking.reshape((numberOfSimsToRun, N))
    # Sum over all alpha's in preparation of getting the average, resulting in a list
    alphaForEachFiberBreaking_total = np.sum(alphaForEachFiberBreaking, axis=0)
    # Divide by the number of sims to get the avg
    alphaForEachFiberBreaking_avg = [alpha_element / numberOfSimsToRun 
                                     for alpha_element in alphaForEachFiberBreaking_total]  

    # Plot the strain curve
    k_N = np.linspace(0.0, 100, N)  # x-vector for plotting
    title = r'Strain curve, $\beta$ = %.2f, $\lambda_{{model}}$ = %.2f %s'%(beta,
                                                            modelLambda, modelLambda_unit)
    xlabel = '%% of broken fibers, of total %.0f'%(N)
    alpha_over_beta = [alpha_break / beta for alpha_break in alphaForEachFiberBreaking_avg]
    plt.figure('Strain Curve')
    plt.title(title)
    plt.plot(k_N, alpha_over_beta, 'k')
    plt.xlabel(xlabel)
    plt.ylabel('$\\alpha$/$\\beta$')
    plt.ylim(0, np.max(alphaForEachFiberBreaking_avg)*1.1/beta)
    ax2 = plt.twinx()
    ax2.set_ylim(0, np.max(alphaForEachFiberBreaking_avg)*1.1* alphaToLoadConversionFactor
                                        / loadName_unit_order)
    ax2.set_ylabel('%s %s'%(loadName, loadName_unit))


    # Plot the breaking order
    plt.figure('Breaking order')
    # This acts as the x axis label, because matshow puts its x labels on top of the figure
    plt.xlabel('Number of fibers broken')
    plt.ylabel('Simulation number')
    # Plot the individual elements of the breakingOrder matrix, and store a handle to the plot
    pictureOfBreakingOrder = plt.imshow(breakingOrder, extent=[0, N, 0, numberOfSimsToRun],
                                        aspect='auto')
    # Create a colorbar for the plot, based on the handle from before
    colorbar = plt.colorbar(pictureOfBreakingOrder) 
    # Set a label to the colorbar
    colorbar.set_label('Breaking of fiber number')

    plt.show()

Мы будем имитировать 100 крючков 1000 раз для стального троса. Попробуйте сами с ДНК и другими параметрами!

In [ ]:
numberOfSimsToRun = 1000
numberOfFibers = 100

In [ ]:
main('steel', numberOfSimsToRun, numberOfFibers)

На первом графике показано значение $\alpha$ (или, что эквивалентно, плотность массы кабеля или электрического поля), необходимое для разрыва следующего крючка. На втором графике можно увидеть, какое число крючков разрывается в зависимости от количества разорванных волокон для каждой из симуляций.

Результаты таковы, как можно было интуитивно ожидать: в начале нарушение является довольно случайным из-за того, что пороговые значения являются случайными. Однако усилие на оставшихся крючках увеличится как за счет увеличения $\alpha$, так и за счет уменьшения количества крючков. Таким образом, через некоторое время соседние крючки будут иметь большую вероятность сломаться. Более того, в какой-то момент мы будем ожидать, что $\alpha$, необходимое для разрыва следующего крючка, уменьшается. То есть максимальное значение $\alpha$ соответствует значению $\alpha$, необходимому для отсоединения кабеля от крыши!

___

## Упражнения и дальнейшая работа
Проверьте код для различных значений $\alpha$ и $\beta$.
 - Что происходит с $\max \alpha_n$, то есть с силой, необходимой для отсоединения кабеля от крыши, или, что эквивалентно, с электрическим полем, необходимым для распутывания полимера, в зависимости от $\beta$?
 - Как выглядит кривая для среднего $\alpha_n$ для больших и малых значений $\beta$? Обратите внимание, что вам может потребоваться диапазон в несколько порядков величин в $\beta$, но помните, что модель предполагает довольно небольшой изгиб кабеля.
 - Сравните типичное количество крючков, которые должны сломаться, прежде чем кабель упадет, для малого и большого $\beta$.  Как можно объяснить эту разницу?
 - Изучите порядок, в котором крючки ломаются более внимательно. Как он меняется с $\beta$?
 
<a id="rsc"></a>
## Resources and further reading

The codes are based on a set of Python scripts originally developed by Håkon Tormodsen Nygård, later modified by Jonas Blomberg Ghini and are available at https://www.numfys.net/media/notebooks/files/FiberBundeModel_Code_170313.zip. The scripts also contain an interactive feature originally developed by Morten Vassvik. Check it out!

Project description in Norwegian, https://www.numfys.net/media/notebooks/files/TMA4320_Biofysikk_og_MedisinskTeknologi_Prosjekt.pdf.

<a>[1]</a> A. Hansen, P. C. Hemmer og S. Pradhan, The Fiber Bundle Model (Wiley-VCH, Berlin, 2015)  
<a>[2]</a> H. Nygård, The Fiber Bundle Bridge Model, Project Thesis, NTNU, 2016.  
<a>[3]</a> S. P. Timoshenko. History of Strength of Materials. With a Brief Account of the History
of Theory of Elasticity and Theory of Structures. Dover Publications, Inc., 1983.
Chap. V, p. 113. isbn: 0-486-61187-6.  
<a>[4]</a> The Engineering Toolbox: Modulus of Elasticity or Young's Modulus - and Tensile Modulus for common Materials  http://www.engineeringtoolbox.com/young-modulus-d_417.html [aquired 04.05.2017]  
<a>[5]</a> Bloom KS. Beyond the code: the mechanical properties of DNA as they relate to mitosis. Chromosoma. 2008;117(2):103-110. doi:10.1007/s00412-007-0138-0.